## import libraries

In [1]:
import asyncio
import math
from math import log, sin, cos, tan, exp, sqrt, pi
import time
from random import randrange
import torch
import numpy as np
from classroom import Classroom
from classroom import Student
from classroom import BytesDataset
from classroom import GutenbergSnippetsDataset
from classroom import GutenbergBitsDataset
from classroom import GutenbergGPT2Dataset
from classroom import MLPLM, MyLM, ABPCNLM
from classroom import TransformerLM
from classroom import AdamW
from classroom import Sonny
from classroom import Floyd
from classroom import Plot
from classroom import Fun
from classroom import Count
from classroom import Sum
from classroom import Diff
from classroom import Log2Sum
from classroom import KalmanFilter1D
from classroom import MedianFilter
from classroom import TwoWindowFilter
from classroom import numel
from classroom import utf8decode, utf8encode, gpt2decode, gpt2encode
from classroom import utf8bitsdecode, utf8bitsencode
from pathlib import Path
import numba

Loading BokehJS ...

## initialize model

In [2]:
if True:
    path = '2021-10-20-0758.pt'

In [3]:
if True:
    model = torch.load(path).to('cuda')

In [ ]:
if False:
    model = (
        MLPLM(
            n_vocab_in=50257,
            n_ctx=16,
            d_model=16,
            d_hidden=16,
            nonlinearity="GELU",
            n_vocab_out=50257).to('cuda'))

In [ ]:
if False:
    model = (
        MyLM(
            n_vocab_in=50257,
            n_ctx=65,
            d_model=64,
            n_layers=1,
            d_hidden=256,
            nonlinearity="GELU",
            p_dropout=0.00,
            n_vocab_out=50257).to('cuda'))

In [ ]:
if True:
    model = (
        ABPCNLM(
            n_vocab_in=50257,
            n_ctx=128,
            d_model=8,
            n_layers=1,
            d_hidden=1024,
            nonlinearity="GELU",
            p_dropout=0.0,
            n_vocab_out=50257).to('cuda'))

In [ ]:
if False:
    model = (
        TransformerLM(
            n_vocab_in=50257,
            n_vocab_out=50257,
            n_ctx=1024,
            d_model=1024,
            d_k=32,
            d_v=32,
            n_heads=32,
            d_hidden=4096,
            n_layers=2,
            p_dropout_embedding=0.1,
            p_dropout_attn_mat=0.01,
            p_dropout_attn_out=0.1,
            p_dropout_mlp=0.01).to('cuda'))

In [4]:
numel(model)

61370073

## initialize student

In [5]:
optimizer = AdamW(parameters=model.named_parameters())
dataset = GutenbergGPT2Dataset()
batch_size = None
example_length = model.n_ctx + 1

student= Student(
    model=model,
    optimizer=optimizer,
    dataset=dataset,
    batch_size=batch_size,
    example_length=example_length)

## schedule hyperparameters

In [78]:
student.batch_size = 1024
student.example_length = 129
for (idx, (pn, p)) in enumerate(student.model.named_parameters()):
    batch_multiplier = 32
    lr_base = 1e-8
    lr = lambda n: lr_base*(n%10000) # *(.5 + sin(pi*n/1000)**2)
    student.optimizer.state[pn]["lr"]           = lambda n: lr(n)
    student.optimizer.state[pn]["beta1"]        = lambda n: 0.9
    student.optimizer.state[pn]["beta2"]        = lambda n: 0.999
    student.optimizer.state[pn]["weight_decay"] = lambda n: 0.001
    student.optimizer.state[pn]["update"]       = lambda n: n%batch_multiplier == 0

## test a single iteration

In [7]:
student.study()

## initialize baseline

In [8]:
if True:
    student.reset_baseline()
    n_of_last_baseline = len(student.times)
    t_start = time.time()
    t_of_last_baseline = 0

## start training

In [9]:
import asyncio
async def train(student):
    while True:
        student.study()
        await asyncio.sleep(1e-4)

In [10]:
training_task = asyncio.create_task(train(student))

## autocomplete

In [12]:
def autocomplete(model, prompt=None, n_generate=512,
                     n_ctx=None, temp=1.0,
                     encode=None, decode=None, output=None):
    Categorical = torch.distributions.Categorical
    if n_ctx is None:
        n_ctx = model.n_ctx
    if encode is None:
        encode = gpt2encode
    if decode is None:
        decode = gpt2decode
    if prompt is None:
        prompt = decode(student.dataset.batch(1, 2*n_ctx, offset=None).tolist()[0])  # kludge
    x = encode(prompt)
    x = x[-n_ctx:]
    prompt = decode(x)
    print(f"=== Prompt ===\n{prompt}\n=== Autocompletion ===\n")

    def sampler(x):
        x = list(x)
        for _ in range(n_generate):
            probs = model.inference(torch.tensor(x, dtype=torch.long, device="cuda").unsqueeze(0)).view(-1)[-model.n_vocab_out:]
            if temp > 0:
                y = Categorical(probs=probs**(1.0/temp)).sample().item()
            else:
                y = torch.argmax(probs).item()
            x = (x + [y])[-n_ctx:]
            if output is not None:
                output.append(y)
            yield y
    result = decode(list(sampler(x)))
    print(result)


In [89]:
autocomplete(model=student.model, temp=0.6, n_ctx=128)

=== Prompt ===
'll
have to find a husband for her. That will cure her of her craze for the
circus."

"A husband for Jeanne, little Jeanne!" Madame Perrault exclaimed in
horror. "She's barely fourteen."

"And in two years she'll be a woman. I was in love with you at fifteen.
Don't you remember? We thought of eloping."

"_Taisez-vous!_" cried Madame Perrault, flushing, and trying not to join
in the laughter that the
=== Autocompletion ===

 upshot, which, all the other, I
couldn't you have ever seen, and don't think about it
was very much for him.

"I don't know, to the appeal, "who is a thing, and
shepher a woman, but I'm not able to do, and, and
if you know. I did not know, and this is a kind of
the bells, and you can see, and I'm afraid of it. I know
you know that I would not have to be the time that I am to be,
and I have not a son of a woman to do, and I want to be
over the wind of the lady from the time.

"I will not do," he said, "and I have had to be a
speech?"

"I don't know," s

## plots

In [14]:
import time
plot_data = {}
lag = 100
X = Fun(Log2Sum(), student.times)
Y = Fun(TwoWindowFilter(lag=lag), student.grades)
Z = Fun(TwoWindowFilter(lag=lag), student.baseline_grades)
plot_data.update({f"grades": (X, Y)})
plot_data.update({f"baseline": (X, Z)})
# Plot(**plot_data)

In [87]:
import time
plot_data_2 = {}
lag = 1000
X = Fun(Count(), student.times)
Y = Fun(lambda x, y: x - y, student.grades, student.baseline_grades)
Y = Fun(TwoWindowFilter(lag=lag), Y.output, aux=Y)
plot_data_2.update({f"improvement": (X, Y)})
Plot(**plot_data_2)

## stats

In [34]:
student.model.n_layers = 2

In [35]:
student.baseline.baseline_model.n_layers = 2

In [88]:
import time

n = len(student.times)-1
t = time.time() - t_start
dn = n - n_of_last_baseline
dt = t - t_of_last_baseline

N = min(1000, dn//2)

y = np.mean(np.array(student.grades[n-N:n]))
y0 = np.mean(np.array(student.baseline_grades[n-N:n]))
dy = y - y0

message = '\n'.join([
    f"batch_size            = {student.batch_size}",
    f"example_length        = {student.example_length}",
    f"100*y                 = {int(y*1e6)/1e4}",
    f"n                     = {n} steps",
    f"t                     = {int(t)} seconds",
    f"n_of_last_baseline    = {n_of_last_baseline} steps",
    f"t_of_last_baseline    = {int(t_of_last_baseline)} seconds",
    f"steps per second      = {dn/dt}",
    f"y0                    = {int(y0*1e6)/1e6}",
    f"dy                    = {int(dy*1e6)/1e6}",
    f"dn                    = {dn}",
    f"dt                    = {dt}",
    f"dy/dn                 = {dy/dn}",
    f"dy/dt                 = {int(1e8 * dy/dt)}e-8 per second",
    f"time to 71            = {(.71-y)/(dy/dt)/3600} hours",
])
print(message)

batch_size            = 1024
example_length        = 129
100*y                 = 62.8203
n                     = 179443 steps
t                     = 37324 seconds
n_of_last_baseline    = 172286 steps
t_of_last_baseline    = 35644 seconds
steps per second      = 4.259470338385815
y0                    = 0.633754
dy                    = -0.005551
dn                    = 7157
dt                    = 1680.2558608055115
dy/dn                 = -7.756478637894196e-07
dy/dt                 = -330e-8 per second
time to 71            = -6.877222807373973 hours


In [79]:
if True:
    student.reset_baseline()
    n_of_last_baseline = len(student.times)-1
    t_of_last_baseline = time.time() - t_start

## save

In [77]:
torch.save(student.model, f=path)

In [ ]:
import asyncio
async def autosave():
    while True:
        await asyncio.sleep(3600)
        torch.save(student.model, f='autosave.pt')
task = asyncio.create_task(autosave())


In [73]:
lyles_constant = 9115131782/14818489608 * log(50257)/log(65536)
lyles_constant

0.6003958292396724

In [74]:
utf8grade = lambda x: 1 - (1 - x)*lyles_constant
grade = .63
print(f"gpt2 grade = {grade}, utf8 grade = {utf8grade(grade)}, bpc = {(1-utf8grade(grade))*8}")

gpt2 grade = 0.63, utf8 grade = 0.7778535431813212, bpc = 1.7771716545494307


## parameter histograms

In [ ]:
histograms = []
for (idx, (pn, p)) in enumerate(student.model.named_parameters()):
    with torch.no_grad():
        print(idx, pn, torch.sqrt(torch.var(p)).item())
        Y, X = np.histogram(p.detach().cpu().numpy(), bins=int(sqrt(torch.numel(p))), density=True)
        print(X.shape, Y.shape)
        histograms.append(Plot(**{f"hist-{idx}": (X.tolist(), Y.tolist())}))

In [ ]:
histograms[4] # 3 7 9 13 15 21 43

## batch-level grade histogram

In [ ]:
Y, X = np.histogram(student.grades[-5000:], bins=256, range=(0,1.0), density=True)
V, U = np.histogram(student.baseline_grades[-5000:], bins=256, range=(0,1.0), density=True)
Plot(**{f"grade-hist": (X, Y), "baseline": (U, V)})

In [ ]:
model.n_ctx, model.d_model, model.d_hidden, model.n_layers

## example-level grade histogram

In [ ]:
def get_graded_examples():
    result = []
    for batch_idx in range(16):
        print(f"batch_idx = {batch_idx}/256")
        x = student.dataset.batch(student.batch_size, student.example_length)
        print(f"orig {x.shape}")
        with torch.no_grad():
            y = student.model(x)
            y = 1.0 - y.cpu().numpy()
            result.append(y)
    data = np.concatenate(result, axis=0)
    result = data.tolist()
    return result

In [ ]:
graded_examples = get_graded_examples()

In [ ]:
len(graded_examples)

In [ ]:
sum(x for x in graded_examples)/len(graded_examples)

In [ ]:
R = (0, 1)
def XYFor(k):
    es = graded_examples
    bins = int(sqrt(len(es)))
    Y, X = np.histogram(es, bins=bins, range=R, density=True)
    return (X, Y)
Plot(**{f"examples-hist-{k}": XYFor(k) for k in [1]})

In [ ]:
ord(' ')

In [ ]:
np.mean(example_grades)

In [ ]:
(1 - 0.7870894884999871)*8

In [ ]:
(1 - 0.8)*8

In [ ]:
(1 - 0.9)*8

In [ ]:
x = np.array([[1,2],[3,4]],dtype=np.uint8)

In [ ]:
np.unpackbits(x, axis=1)